In [1]:
import json
import datetime as dt
import urllib.request
import pandas as pd
import numpy as np

from sqlalchemy import Column, Integer, Float, String
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy import inspect
from sqlalchemy import select

In [2]:
requestURL = "https://eodhistoricaldata.com/api/eod/"
myEodKey = "5ba84ea974ab42.45160048"
startDate = dt.datetime(2017,12,31)
endDate = dt.datetime(2019,5,3)

In [3]:
stock_list = pd.read_csv("PairTrading.csv")

In [4]:
stock_list.head()

,Stock1,Stock2
0,AAPL,HPQ
1,APC,CHK
2,APC,DVN
3,AXP,COF
4,BAC,JPM


In [5]:
stock1 = []
for stock in stock_list["Stock1"]:
    if stock not in stock1:
        stock1.append(stock)

stock2 = []
for stock in stock_list["Stock2"]:
    if stock not in stock2:
        stock2.append(stock)

In [6]:
def get_daily_data(symbol, start=startDate, end=endDate, requestType=requestURL, apiKey=myEodKey):
    symbolURL = str(symbol) + ".US?"
    startURL = "from=" + str(start)
    endURL = "to=" + str(end)
    apiKeyURL = "api_token=" + myEodKey
    completeURL = requestURL + symbolURL + startURL + '&' + endURL + '&' + apiKeyURL + '&period=d&fmt=json'
    print(completeURL)
    with urllib.request.urlopen(completeURL) as req:
        data = json.load(req)
        return data
    

In [7]:
def create_pair1_table(name, metadata, engine):
	tables = metadata.tables.keys()
	if name not in tables:
		table = Table(name, metadata, 
					Column('symbol_1', String(50), primary_key=True, nullable=False),
					Column('date', String(50), primary_key=True, nullable=False),
					Column('open_1', Float, nullable=False),
					Column('high_1', Float, nullable=False),
					Column('low_1', Float, nullable=False),
					Column('close_1', Float, nullable=False),
                    Column('adjusted_close_1', Float, nullable=False),
					Column('volume_1', Integer, nullable=False))
		table.create(engine)
		return table

def create_pair2_table(name, metadata, engine):
	tables = metadata.tables.keys()
	if name not in tables:
		table = Table(name, metadata, 
					Column('symbol_2', String(50), primary_key=True, nullable=False),
					Column('date', String(50), primary_key=True, nullable=False),
					Column('open_2', Float, nullable=False),
					Column('high_2', Float, nullable=False),
					Column('low_2', Float, nullable=False),
					Column('close_2', Float, nullable=False),
					Column('adjusted_close_2', Float, nullable=False),
					Column('volume_2', Integer, nullable=False))
		table.create(engine)
		return table

    
def populate_stock1_data(tickers, metadata, engine, table_name):
    conn = engine.connect()
    table = metadata.tables[table_name]
    for ticker in tickers:
        stock = get_daily_data(ticker)
        #print(stock)
        for stock_data in stock:
            #print(k, v)
            trading_date = stock_data['date']
            trading_open = stock_data['open']
            trading_high = stock_data['high']
            trading_low = stock_data['low']
            trading_close = stock_data['close']
            trading_adjusted_close = stock_data['adjusted_close']
            trading_volume = stock_data['volume']
            insert_st = table.insert().values(symbol_1=ticker, date=trading_date,
					open_1 = trading_open, high_1 = trading_high, low_1 = trading_low,
					close_1 = trading_close, adjusted_close_1 = trading_adjusted_close, 
                       volume_1 = trading_volume)
            conn.execute(insert_st)

def populate_stock2_data(tickers, metadata, engine, table_name):
    conn = engine.connect()
    table = metadata.tables[table_name]
    for ticker in tickers:
        stock = get_daily_data(ticker)
        #print(stock)
        for stock_data in stock:
            #print(k, v)
            trading_date = stock_data['date']
            trading_open = stock_data['open']
            trading_high = stock_data['high']
            trading_low = stock_data['low']
            trading_close = stock_data['close']
            trading_adjusted_close = stock_data['adjusted_close']
            trading_volume = stock_data['volume']
            insert_st = table.insert().values(symbol_2=ticker, date=trading_date,
					open_2 = trading_open, high_2 = trading_high, low_2 = trading_low,
					close_2 = trading_close, adjusted_close_2 = trading_adjusted_close, 
                       volume_2 = trading_volume)
            conn.execute(insert_st)



In [8]:
def create_PairPrices_table(name, metadata, engine):
	tables = metadata.tables.keys()
	if name not in tables:
		table = Table(name, metadata, 
					Column('symbol_1', String(50), primary_key=True, nullable=False),
					Column('symbol_2', String(50), primary_key=True, nullable=False),
					Column('date', String(50), primary_key=True, nullable=False),
					Column('open_1', Float, nullable=False),
					Column('close_1', Float, nullable=False),
					Column('open_2', Float, nullable=False),
					Column('close_2', Float, nullable=False))
		table.create(engine)
	return table

def populate_PairsPrices_data(stock_list, metadata, engine, table_name,stock1_table,stock2_table):
    conn = engine.connect()
    table = metadata.tables[table_name]
    
    for row in np.array(stock_list):
        ticker_1 =row[0]
        ticker_2 =row[1]
        
        s = select([stock1_table.c.date,stock1_table.c.open_1,stock1_table.c.close_1]).where(stock1_table.c.symbol_1==ticker_1)
        result = conn.execute(s)
        res_1 = result.fetchall()
        
        s = select([stock2_table.c.open_2,stock2_table.c.close_2]).where(stock2_table.c.symbol_2==ticker_2)
        result = conn.execute(s)
        res_2 = result.fetchall()
        
        for i,item in enumerate(res_1):
            insert_st = table.insert().values(symbol_1=ticker_1, symbol_2=ticker_2,
                                              date = res_1[i][0] , open_1 = res_1[i][1] ,close_1 = res_1[i][2] ,
                                              open_2 = res_2[i][0],close_2 = res_2[i][1])
            conn.execute(insert_st)


In [9]:
def create_Pairs_table(name, metadata, engine):
	tables = metadata.tables.keys()
	if name not in tables:
		table = Table(name, metadata, 
					Column('symbol_1', String(50), primary_key=True, nullable=False),
					Column('symbol_2', String(50), primary_key=True, nullable=False),
					Column('volatility', Float, nullable=False),
					Column('profit_loss', Float, nullable=False))
		table.create(engine)
	return table
        
        
def populate_Pairs_data(stock_list, metadata, engine, table_name,stock1_table,stock2_table):
    conn = engine.connect()
    table = metadata.tables[table_name]
    

    for row in np.array(stock_list):
        ticker_1 =row[0]
        ticker_2 =row[1]

        
        s = select([stock1_table.c.close_1]).where(stock1_table.c.symbol_1==ticker_1).where(stock1_table.c.date<="2018-12-31").where(stock1_table.c.date>="2018-01-01")
        result = conn.execute(s)
        close_1 = result.fetchall()
        close_1 = np.array(close_1).flatten()

        
        s = select([stock2_table.c.close_2]).where(stock2_table.c.symbol_2==ticker_2).where(stock2_table.c.date<="2018-12-31").where(stock2_table.c.date>="2018-01-01")
        result = conn.execute(s)
        close_2 = result.fetchall()
        close_2 = np.array(close_2).flatten()
        
        volatility = np.std(close_1/close_2)
        insert_st = table.insert().values(symbol_1=ticker_1, symbol_2=ticker_2,
                                            volatility = volatility,profit_loss = 0.0)
        
        conn.execute(insert_st)
        
#def populate_Pairs_data(tickers, metadata, engine, table_name):

            
        
    


In [10]:
        
def create_Trades_table(name, metadata, engine):
	tables = metadata.tables.keys()
	if name not in tables:
		table = Table(name, metadata, 
					Column('symbol_1', String(50), primary_key=True, nullable=False),
                    Column('symbol_2', String(50), primary_key=True, nullable=False),
					Column('date', String(50), primary_key=True, nullable=False),
					Column('profit_loss_d', Float, nullable=False))
		table.create(engine)
	return table
     

In [11]:
def populate_Trades_data(tickers, metadata, engine, table_name,Pairs_table,stock1_table,stock2_table,k):
   
    conn = engine.connect()
    table = metadata.tables[table_name]
    
    s = select([Pairs_table.c.volatility])
    result = conn.execute(s)
    volatility = result.fetchall()
    volatility = np.array(volatility).flatten()
    
    s = select([stock1_table.c.date]).where(stock1_table.c.symbol_1=="AAPL").where(stock1_table.c.date<="2019-01-31").where(stock1_table.c.date>="2018-12-31")
    result = conn.execute(s)
    dates= result.fetchall()
    dates = np.array(dates).flatten()
    
    
    for i,row in enumerate(np.array(stock_list)):
        ticker_1 =row[0]
        ticker_2 =row[1]

        s = select([stock1_table.c.open_1,stock1_table.c.close_1]).where(stock1_table.c.symbol_1=="AAPL").where(stock1_table.c.date<="2019-1-31").where(stock1_table.c.date>="2018-12-31")
        result = conn.execute(s)
        result = result.fetchall()
        open_1,close_1 = np.array(result).transpose()

        s = select([stock2_table.c.open_2,stock2_table.c.close_2]).where(stock2_table.c.symbol_2==ticker_2).where(stock2_table.c.date<="2019-1-31").where(stock2_table.c.date>="2018-12-31")
        result = conn.execute(s)
        result = result.fetchall()
        open_2,close_2 = np.array(result).transpose()
        
        condition = close_1[:-1]/close_2[:-1]-open_1[1:]/open_2[1:]-k*volatility[i]
        PnL_sum = 0
        
        for j,date in enumerate(dates):
            if j ==0:
                continue
            if (condition[j-1]>0):
                n1 = -10000
            if(condition[j-1]<0):
                n1 = 10000
                
            n2 = - np.floor(n1*(open_1[j]/open_2[j]))
            PnL = n1*(close_1[j]-open_1[j]) + n2*(close_2[j]-open_2[j]) 
            
            insert_st = table.insert().values(symbol_1=ticker_1, symbol_2=ticker_2,
                                            date = date,profit_loss_d = PnL)
            conn.execute(insert_st)
        
            PnL_sum += PnL
        print(PnL)
        table_2 = metadata.tables["Pairs_table"]
        
        stmt = table_2.update().values(profit_loss = PnL_sum).where(table_2.c.symbol_1==ticker_1).where(table_2.c.symbol_2==ticker_2)
        conn.execute(stmt)
                
                
        
        
    

def clear_a_table(table_name, metadata, engine):
    conn = engine.connect()
    table = metadata.tables[table_name]
    delete_st = table.delete()
    conn.execute(delete_st)

    


def execute_sql_statement(sql_st, engine):
    result = engine.execute(sql_st)
    return result

def build_pair_trading_model():
    return 0

In [12]:
metadata = MetaData()
engine =  create_engine('sqlite:///:memory:')
conn = engine.connect()

In [13]:
stock1_table = create_pair1_table("Pair1Stocks", metadata, engine)

In [14]:
populate_stock1_data(stock1, metadata, engine,"Pair1Stocks")

https://eodhistoricaldata.com/api/eod/AAPL.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/APC.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/AXP.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/BAC.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/BHP.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/CAT.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/CHK.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&pe

In [15]:
stock2_table = create_pair2_table("Pair2Stocks", metadata, engine)
populate_stock2_data(stock2, metadata, engine,"Pair2Stocks")

https://eodhistoricaldata.com/api/eod/HPQ.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/CHK.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/DVN.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/COF.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/JPM.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/FCX.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&period=d&fmt=json
https://eodhistoricaldata.com/api/eod/DE.US?from=2017-12-31 00:00:00&to=2019-05-03 00:00:00&api_token=5ba84ea974ab42.45160048&peri

In [16]:
PairPrices_table = create_PairPrices_table("PairPrices", metadata, engine)
populate_PairsPrices_data(stock_list, metadata, engine, "PairPrices",stock1_table,stock2_table)

In [17]:
Pairs_table = create_Pairs_table("Pairs_table", metadata, engine)#volatility,PnL
populate_Pairs_data(stock_list, metadata, engine, "Pairs_table",stock1_table,stock2_table)

In [18]:
k = 1
Trades_table = create_Trades_table("Trades_table", metadata, engine)


In [19]:
populate_Trades_data(stock_list, metadata, engine,"Trades_table",Pairs_table,stock1_table,stock2_table,k)

16020.079999999703
43120.82999999975
73158.35999999994
-3114.2100000002065
1854.329999999786
-57345.56000000018
-12754.82000000009
73158.35999999994
-9106.600000000282
-4216.280000000333
8334.71999999975
-19448.22000000004
8801.29999999996
34265.89999999977
34265.89999999977
9005.669999999931
-6646.700000000167
-8470.320000000258
-11865.83999999999
-2439.300000000182
-2221.2800000002862
-8470.320000000258
-27739.320000000316
24966.44999999989


In [20]:
s=select([Pairs_table])
result = conn.execute(s)
result.fetchall()

[('AAPL', 'HPQ', 0.5553570579532815, -29696.490000000478),
 ('APC', 'CHK', 2.777387301509632, -111562.48000000039),
 ('APC', 'DVN', 0.17100667356242524, -76316.6500000003),
 ('AXP', 'COF', 0.0969850938625831, -72201.87000000065),
 ('BAC', 'JPM', 0.009124729046404547, -37388.7300000008),
 ('BHP', 'FCX', 0.6071847036602649, -110788.6699999997),
 ('CAT', 'DE', 0.058142729357438465, -88041.4400000004),
 ('CHK', 'DVN', 0.012774344636429496, -87365.31000000027),
 ('COP', 'CVX', 0.05274113303178558, 22670.70499999973),
 ('CS', 'DB', 0.10532381402416413, -67651.91999999974),
 ('CSX', 'NSC', 0.016273741089269617, -142498.0),
 ('CVX', 'XOM', 0.053188692413880884, 20660.009999999307),
 ('DAL', 'UAL', 0.07489218195313094, -66522.47999999927),
 ('DSX', 'GNK', 0.05266301633153175, -88573.30999999991),
 ('EGLE', 'GNK', 0.07368092729346519, -88573.30999999991),
 ('GG', 'GLD', 0.012487072768956125, 81975.6799999996),
 ('GG', 'KGC', 0.21790516868738513, 76911.6499999987),
 ('GG', 'NEM', 0.02619561178822

In [21]:
conn = engine.connect()
s = select([stock1_table.c.open_1,stock1_table.c.close_1]).where(stock1_table.c.symbol_1=="AAPL").where(stock1_table.c.date=="2019-01-02")
result = conn.execute(s)
result= result.fetchall()
open_1_d1 =result[0][0]
close_1_d1 = result[0][1]

In [22]:
result

[(154.89, 157.92)]

In [25]:
def real_time_trading():
    print("Real time trade start:")
    print("Please enter parameter k:")
    k = float(input())
    
    #input the tickers for the pair
    print("Please input ticker for the first stock of the pair:")
    ticker_1 = input()
    print("Please enter a open price for it")
    open_1_d2 = float(input())    
    print("Please enter a close price for it")
    close_1_d2 = float(input())
    
    
    print("Please input ticker for the second stock of the pair:")
    ticker_2 = input()
    print("Please enter a open price for it")
    open_2_d2 = float(input())    
    print("Please enter a close price for it")
    close_2_d2 = float(input())
    
    
    print("Which day you want to choose as yesterday?(Please enter as yyyy-mm-dd)")
    date_2 = input()
    
    conn = engine.connect()
    
    #get selected date open and close prices
    s = select([stock1_table.c.open_1,stock1_table.c.close_1]).where(stock1_table.c.symbol_1==ticker_1).where(stock1_table.c.date==date_2)
    result = conn.execute(s)
    result= result.fetchall()
    open_1_d1 =result[0][0]
    close_1_d1 = result[0][1]
    
    s = select([stock2_table.c.open_2,stock2_table.c.close_2]).where(stock2_table.c.symbol_2==ticker_2).where(stock2_table.c.date==date_2)
    result = conn.execute(s)
    result= result.fetchall()
    open_2_d1 =  result[0][0]
    close_2_d1 = result[0][1]
    
    s=select([Pairs_table.c.volatility]).where(Pairs_table.c.symbol_1 == ticker_1).where(Pairs_table.c.symbol_2 == ticker_2)
    result = conn.execute(s)
    volatility = result.fetchall()
    volatility = volatility[0][0] 

    if((close_1_d1/close_2_d1 - open_1_d2/open_2_d2)>= (k * volatility)):
        N1 = -10000
        print("")
        print("We accomplish result shown below:")
        print("Short this pair")
        print("Historical volatility:{}".format(volatility))
        print("Short {s1} of 10000 shares".format(s1 = ticker_1,s2 = N1))
        N2 = -N1 * (open_1_d2/open_2_d2)
        print("long {s1} of {s2} shares".format(s1 =ticker_2 ,s2 = N2))
        
        PnL =  N1 * (close_1_d2 - open_1_d2) + N2 * (close_2_d2 - open_2_d2)
        
        print("Overall PnL :{}".format(PnL))
        
    else:
        N1 = 10000
        print("")
        print("We accomplish result shown below:")
        print("Long this pair")
        print("Historical volatility:{}".format(volatility))
        print("Long {s1} of 10000 shares".format(s1 = ticker_1,s2 = N1))
        N2 = -N1 * (open_1_d2/open_2_d2)
        print("short {s1} of {s2} shares".format(s1 =ticker_2 ,s2 = N2))
        
        PnL =  N1 * (close_1_d2 - open_1_d2) + N2 * (close_2_d2 - open_2_d2)
        
        print("Overall PnL :{}".format(PnL))
        
        return 12138
        
        
        
    

In [26]:
real_time_trading()

Real time trade start:
Please enter parameter k:
1
Please input ticker for the first stock of the pair:
AAPL
Please enter a open price for it
5
Please enter a close price for it
6
Please input ticker for the second stock of the pair:
HPQ
Please enter a open price for it
5
Please enter a close price for it
6
Which day you want to choose as yesterday?(Please enter as yyyy-mm-dd)
2019-01-02

We accomplish result shown below:
Short this pair
Historical volatility:0.5553570579532815
Short AAPL of 10000 shares
long HPQ of 10000.0 shares
Overall PnL :0.0
